# Textklassificeringsuppgift

Som vi nämnt kommer vi att fokusera på en enkel textklassificeringsuppgift baserad på **AG_NEWS**-datasetet, där nyhetsrubriker ska klassificeras i en av fyra kategorier: Världen, Sport, Ekonomi och Vetenskap/Teknik.

## Datasetet

Detta dataset är inbyggt i [`torchtext`](https://github.com/pytorch/text)-modulen, så vi kan enkelt komma åt det.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Här innehåller `train_dataset` och `test_dataset` samlingar som returnerar par av etikett (klassnummer) och text respektive, till exempel:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Så, låt oss skriva ut de första 10 nya rubrikerna från vår datamängd:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Eftersom dataset är iteratorer, om vi vill använda datan flera gånger måste vi konvertera den till en lista:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenisering

Nu behöver vi konvertera text till **siffror** som kan representeras som tensorer. Om vi vill ha ordnivårepresentation måste vi göra två saker:
* använda **tokenizer** för att dela upp texten i **token**
* bygga ett **ordförråd** av dessa token.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Genom att använda ordförråd kan vi enkelt koda vår tokeniserade sträng till en uppsättning siffror:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Påsmodell för textrepresentation

Eftersom ord representerar betydelse kan vi ibland förstå innebörden av en text bara genom att titta på de enskilda orden, oavsett deras ordning i meningen. Till exempel, när vi klassificerar nyheter, är ord som *väder*, *snö* sannolikt att indikera *väderprognos*, medan ord som *aktier*, *dollar* skulle peka mot *finansiella nyheter*.

**Påsmodell** (BoW) vektorrepresentation är den mest använda traditionella vektorrepresentationen. Varje ord är kopplat till ett vektorindex, och varje element i vektorn innehåller antalet förekomster av ett ord i ett givet dokument.

![Bild som visar hur en påsmodell-vektorrepresentation lagras i minnet.](../../../../../translated_images/sv/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Du kan också tänka på BoW som en summa av alla enskilda one-hot-kodade vektorer för individuella ord i texten.

Nedan är ett exempel på hur man genererar en påsmodellrepresentation med hjälp av Scikit Learn python-biblioteket:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

För att beräkna bag-of-words-vektorn från vektorrepresentationen av vår AG_NEWS-dataset kan vi använda följande funktion:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Observera:** Här använder vi den globala variabeln `vocab_size` för att ange standardstorleken på ordförrådet. Eftersom ordförrådets storlek ofta är ganska stor kan vi begränsa storleken på ordförrådet till de mest frekventa orden. Försök att sänka värdet på `vocab_size` och köra koden nedan, och se hur det påverkar noggrannheten. Du bör förvänta dig en viss minskning i noggrannhet, men inte dramatisk, i utbyte mot högre prestanda.


## Träna BoW-klassificerare

Nu när vi har lärt oss att bygga en Bag-of-Words-representation av vår text, låt oss träna en klassificerare ovanpå den. Först behöver vi konvertera vår dataset för träning på ett sådant sätt att alla positionsvektorrepresentationer omvandlas till Bag-of-Words-representation. Detta kan uppnås genom att skicka funktionen `bowify` som parametern `collate_fn` till standard torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Nu ska vi definiera ett enkelt klassificeringsneuronätverk som innehåller ett linjärt lager. Storleken på ingångsvektorn är lika med `vocab_size`, och utgångsstorleken motsvarar antalet klasser (4). Eftersom vi löser en klassificeringsuppgift är den slutliga aktiveringsfunktionen `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Nu ska vi definiera standardträningsloopen i PyTorch. Eftersom vår dataset är ganska stor, kommer vi för undervisningssyfte att träna endast i en epok, och ibland till och med mindre än en epok (genom att ange parametern `epoch_size` kan vi begränsa träningen). Vi kommer också att rapportera ackumulerad träningsnoggrannhet under träningen; frekvensen för rapportering anges med parametern `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams och N-Grams

En begränsning med en bag-of-words-metod är att vissa ord ingår i flerordsuttryck. Till exempel har ordet 'hot dog' en helt annan betydelse än orden 'hot' och 'dog' i andra sammanhang. Om vi alltid representerar orden 'hot' och 'dog' med samma vektorer kan det förvirra vår modell.

För att hantera detta används ofta **N-gram-representationer** i metoder för dokumentklassificering, där frekvensen av varje ord, tvåords- eller treordsuttryck är en användbar egenskap för att träna klassificerare. I en bigram-representation, till exempel, lägger vi till alla ordpar i vokabulären, utöver de ursprungliga orden.

Nedan är ett exempel på hur man genererar en bigram bag-of-words-representation med hjälp av Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Den största nackdelen med N-gram-metoden är att ordförrådet börjar växa extremt snabbt. I praktiken behöver vi kombinera N-gram-representationen med några tekniker för dimensionsreduktion, såsom *embeddings*, vilket vi kommer att diskutera i nästa avsnitt.

För att använda N-gram-representation i vår **AG News**-dataset behöver vi skapa ett speciellt ngram-ordförråd:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Vi skulle kunna använda samma kod som ovan för att träna klassificeraren, men det skulle vara väldigt minnesineffektivt. I nästa avsnitt kommer vi att träna en bigram-klassificerare med hjälp av embeddings.

> **Note:** Du kan endast behålla de ngram som förekommer i texten fler gånger än det angivna antalet. Detta säkerställer att sällsynta bigram utesluts och minskar dimensionen avsevärt. För att göra detta, ställ in parametern `min_freq` till ett högre värde och observera hur vokabulärens längd förändras.


## Termfrekvens-Invers Dokumentfrekvens (TF-IDF)

I BoW-representationen vägs ords förekomster lika, oavsett vilket ord det är. Det är dock uppenbart att frekventa ord, som *en*, *i*, etc., är mycket mindre viktiga för klassificering än specialiserade termer. Faktum är att i de flesta NLP-uppgifter är vissa ord mer relevanta än andra.

**TF-IDF** står för **termfrekvens–invers dokumentfrekvens**. Det är en variant av bag of words, där man istället för ett binärt 0/1-värde som indikerar förekomsten av ett ord i ett dokument, använder ett flyttalsvärde som är relaterat till frekvensen av ordets förekomst i korpusen.

Mer formellt definieras vikten $w_{ij}$ för ett ord $i$ i dokumentet $j$ som:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
där
* $tf_{ij}$ är antalet förekomster av $i$ i $j$, dvs. det BoW-värde vi sett tidigare
* $N$ är antalet dokument i samlingen
* $df_i$ är antalet dokument som innehåller ordet $i$ i hela samlingen

TF-IDF-värdet $w_{ij}$ ökar proportionellt med antalet gånger ett ord förekommer i ett dokument och justeras efter antalet dokument i korpusen som innehåller ordet, vilket hjälper till att kompensera för det faktum att vissa ord förekommer oftare än andra. Till exempel, om ordet förekommer i *varje* dokument i samlingen, $df_i=N$, och $w_{ij}=0$, och dessa termer skulle helt ignoreras.

Du kan enkelt skapa TF-IDF-vektorisering av text med hjälp av Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Slutsats

Även om TF-IDF-representationer ger frekvensvikt till olika ord, kan de inte representera betydelse eller ordning. Som den berömda lingvisten J. R. Firth sa år 1935: "Den fullständiga betydelsen av ett ord är alltid kontextuell, och ingen studie av betydelse utan kontext kan tas på allvar." Senare i kursen kommer vi att lära oss hur man fångar kontextuell information från text med hjälp av språkmodellering.



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiserade översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
